# 第一次作业-预测PM2.5

In [ ]:
#萌新徒手抓瞎

import numpy as np
import pandas as pd

data=pd.read_csv('train.csv')
data=data[data["object-name"]=="PM2.5"]
data[["1","2"]][:5]

## 扒作业答案源码

In [47]:
import sys
import numpy as np
import pandas as pd
import csv

In [48]:
raw_data=np.genfromtxt("train.csv",delimiter=',') #读入文件
data=raw_data[1:,2:] #略去文件头和左侧属性名
where_are_NaNs=np.isnan(data)
data[where_are_NaNs]=0 #将无效值赋为零

month_to_data={} #这个dict有三个维度，月份、污染物和时间

for month in range(12): #数据中包含12个月
    sample=np.empty(shape=(18,480))
    for day in range(20): #每个月内含20天
        for hour in range(24): #污染物有18种，因此每过一天就乘以18
            sample[:,day*24+hour]=data[18*(month*20+day):18*(month*20+day+1),hour]
    month_to_data[month]=sample

In [49]:
x=np.empty(shape=(12*471,18*9),dtype=float)
y=np.empty(shape=(12*471,1),dtype=float)

for month in range(12):
    for day in range(20):
        for hour in range(24):
            if day==19 and hour>14: #471=12*20-10+1
                continue
            
            #reshape(a,b)其中一个参数为-1意味着自适应另一维度来调整，如(1,-1)即变为一行
            #x是取连续9小时内18种污染物的数值作为原始数据，且化为一行进行储存。两个维度分别是时间和污染物。
            #y是取对应的第十小时PM2.5的数值作为结果
            x[month*471+day*24+hour,:]=month_to_data[month][:,day*24+hour:day*24+hour+9].reshape(1,-1)
            y[month*471+day*24+hour,0]=month_to_data[month][9,day*24+hour+9] #PM2.5在第九行

            #总结一下上面这样处理的合理之处：所有连续9小时都被囊括，包括了跨天的时间段，最大化利用数据
            #一切都归结于reshape的机智操作，赞！
            
# pd.DataFrame(x)
#以前两行为例进行解释
#第0行-第一种污染物从0点开始连续九小时的数值-第二种污染物从0点开始连续九小时的数值-
#第1行-第一种污染物从1点开始连续九小时的数值-第二种污染物从1点开始连续九小时的数值-
#以此类推就得到了训练用的dataset

In [50]:
mean=np.mean(x,axis=0) #dataset中每一列都是同种污染物的数值
std=np.std(x,axis=0)

for i in range(x.shape[0]): #x.shape[0]=12*471
    for j in range(x.shape[1]): #x.shape[1]=18*9
        x[i][j]=(x[i][j]-mean[j])/(std[j]+0.0001) #归一化处理

In [51]:
dim=x.shape[1]+1
w=np.zeros(shape=(dim,1))
x=np.concatenate((np.ones((x.shape[0],1)),x),axis=1).astype(float) #左侧拼接bias项所对应的常数1
learning_rate=np.array([[200]]*dim)
adagrad_sum=np.zeros(shape=(dim,1))

#下面就是进行Gradient Descent
#矢量化操作需要常记的一些点
#1.dataset中的实例作为行，即一行一个实例，然后参数作为列，或者调转，最好形成固定习惯
#2.记住计算平方就是A.T@A，即转置后直接矩阵乘法
#3.如果直接在dataset左侧拼接常数1，那么各种操作都会方便许多

for T in range(100000):
    if T%5000==0:
        print("T=",T)
        print("Loss:",np.sqrt(np.sum(np.power(x.dot(w)-y,2))/x.shape[0]))
    gradient=(-2)*np.transpose(x).dot(y-x.dot(w))
    adagrad_sum+=gradient**2
    w=w-learning_rate/(np.sqrt(adagrad_sum)+0.0005)*gradient

np.save('weight.npy',w) #训练得到的权重存储在特定后缀文件.npy

T= 0
Loss: 27.071214829194115
(10, 5750)
T= 5000
Loss: 5.737230072636322
(163, 1)
T= 10000
Loss: 5.683826252203138
(163, 1)
T= 15000
Loss: 5.681715033739906
(163, 1)
T= 20000
Loss: 5.681030482545131
(163, 1)
T= 25000
Loss: 5.6805853353390665
(163, 1)
T= 30000
Loss: 5.680273034497946
(163, 1)
T= 35000
Loss: 5.680052459541523
(163, 1)


KeyboardInterrupt: 

In [121]:
w=np.load('weight.npy')
test_raw_data=np.genfromtxt('test.csv',delimiter=',')
test_data=test_raw_data[1:,2:]
where_are_NaNs=np.isnan(test_data)
test_data[where_are_NaNs]=0

In [122]:
test_x=np.empty(shape=(240,18*9),dtype=float)

for i in range(240):
    test_x[i,:]=test_data[i*18:(i+1)*18,:].reshape(1,-1)
    
mean=np.mean(test_x,axis=0)
std=np.std(test_x,axis=0)

for i in range(test_x.shape[0]):
    for j in range(test_x.shape[1]):
        if not std[j]==0:
            test_x[i][j]=(test_x[i][j]-mean[j])/std[j]
            
test_x=np.concatenate((np.ones(shape=(test_x.shape[0],1)),test_x),axis=1).astype(float)
answer=test_x.dot(w)
answer

ValueError: shapes (240,163) and (10,1) not aligned: 163 (dim 1) != 10 (dim 0)

In [ ]:
f=open("ans.csv","w+")
ws=csv.writer(f)
title=['id','value']
ws.writerow(title)

for i in range(240):
    content=['id_'+str(i),answer[i][0]]
    ws.writerow(content)

## z君独立尝试

In [117]:
import numpy as np
import sys
import csv

data_raw=np.genfromtxt("train.csv",delimiter=',')
data=data_raw[1:,2:]
where_are_NaNs=np.isnan(data)
data[where_are_NaNs]=0

In [118]:
con_hour_data=[]

for i in range(240):
    con_hour_data.append(data[i*18+9,:])

con_hour_data=np.array(con_hour_data).reshape(1,-1)

x_train=np.zeros((5750,9)) #这里需要非常注意，如果有列向量之类的，不要使用np.array方法进行转换，而是直接用np.zeros方法创建！
y_train=np.zeros((5750,1))

for i in range(con_hour_data.shape[1]-10):
    x_train[i,0:9]=con_hour_data[0,i:i+9]
    y_train[i]=con_hour_data[0,i+9]

In [119]:
mean=np.mean(x_train,axis=1)
std=np.std(x_train,axis=1)

# print(mean.shape)

for i in range(x_train.shape[0]):
    x_train[i]=(x_train[i]-mean[i])/(std[i]+0.0001)

In [120]:
w=np.zeros((10,1))
x_train=np.concatenate((np.ones((x_train.shape[0],1)),x_train),axis=1).astype(float)

learning_rate=np.array([[100]]*10)
adagrad=np.zeros((10,1))

for T in range(100000):
    if T%5000==0:
        print("T:%d" % T)
        print("Loss:%f" % (np.sqrt(np.sum(np.power(y_train-x_train@w,2))/x_train.shape[0])))
    
    gradient=(-2)*np.transpose(x_train)@(y_train-x_train@w)
    adagrad+=gradient**2
    
    w-=learning_rate/np.sqrt(adagrad+0.00001)*gradient
    
np.save("weight.npy",w)

T:0
Loss:27.128038
T:5000
Loss:15.025482
T:10000
Loss:15.025482
T:15000
Loss:15.025482
T:20000
Loss:15.025482
T:25000
Loss:15.025482
T:30000
Loss:15.025482
T:35000
Loss:15.025482
T:40000
Loss:15.025482
T:45000
Loss:15.025482
T:50000
Loss:15.025482
T:55000
Loss:15.025482
T:60000
Loss:15.025482
T:65000
Loss:15.025482
T:70000
Loss:15.025482
T:75000
Loss:15.025482
T:80000
Loss:15.025482
T:85000
Loss:15.025482
T:90000
Loss:15.025482
T:95000
Loss:15.025482
